In [139]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
from __future__ import division

In [6]:
# Load Original Data
df_train = pd.read_csv('../../data/beer/Train.csv')
df_valid = pd.read_csv('../../data/beer/Valid.csv')
df_test = pd.read_csv('../../data/beer/Test.csv')
key_phrase = pd.read_csv('../../data/beer/KeyPhrases.csv')

In [3]:
# Load U-I Data 
rtrain = load_npz("../../data/beer/Rtrain.npz")
rvalid = load_npz("../../data/beer/Rvalid.npz")
rtest = load_npz("../../data/beer/Rtest.npz")

In [4]:
rtrain

<6370x3668 sparse matrix of type '<type 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [8]:
df_train.head()

,beer/ABV,beer/name,beer/style,review/overall,review/text,review/timeUnix,user/profileName,Binary,UserIndex,ItemIndex,review,conca_review,keyVector,keyphrases_indices_length
0,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,Caldera- Pale Ale\t\tPicked up at Grape & Gour...,1251327677,NJpadreFan,0,1767,659,"['caldera', 'pale', 'ale', 'picked', 'grape', ...",caldera pale ale picked grape gourmet vb ligh...,"[1, 7, 12, 14, 16, 29, 31, 34, 39, 40, 42, 45]",12
1,5.5,Caldera Pale Ale,American Pale Ale (APA),2.5,12 oz. can poured into a pint glass. I really ...,1250928902,vacax,0,6159,659,"['oz', 'poured', 'pint', 'glass', 'really', 'l...",oz poured pint glass really like beer like pa...,"[1, 16, 18, 20, 24, 39, 42, 43, 45, 55]",10
2,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,A: Pours a cloudy orange with a crisp white tw...,1249866208,d0ggnate,0,3516,659,"['pours', 'cloudy', 'orange', 'crisp', 'white'...",pours cloudy orange crisp white two finger he...,"[12, 36, 39, 42]",4
3,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,Received this can through Beer Powerball. I am...,1249847121,babyhobbes,1,2893,659,"['received', 'beer', 'powerball', 'really', 's...",received beer powerball really sorry nt know ...,"[12, 42]",2
4,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,355ml can bought from Beersofeurope.com...\t\t...,1249556277,mdagnew,1,4858,659,"['bought', 'beersofeurope', 'com', 'poured', '...",bought beersofeurope com poured golden yellow...,"[1, 5, 7, 12, 14, 16, 18, 19, 26, 31, 34, 40, ...",18


In [7]:
# Models
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
#         if binary:
        vals.extend(np.array([1]*len(key_vector)))
#         else:
#             vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


def predict(matrix_train, k, similarity, item_similarity_en = False):
    """
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]


# U_K * U_U = U_K

In [99]:
# U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (6370,75))
U_K = load_npz('../../data/beer/U_K.npz')

In [10]:
U_U_fromUI = train(rtrain)
U_U_fromUK = train(U_K)

In [11]:
k = 10
testres = predict(U_K, k, U_U_fromUI, item_similarity_en=False)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:02<00:00, 2489.25it/s]


### Normalization

In [17]:
import numpy as np
from sklearn.preprocessing import normalize

x = np.random.rand(1000)*10
norm1 = x / np.linalg.norm(x)
norm2 = normalize(x[:,np.newaxis], axis=0).ravel()
print np.all(norm1 == norm2)
# True

True


## Direct Multiplication of U_K matrices

In [15]:
testres[0]

array([ 84.00923042, 215.8694369 , 105.07053825,   8.86858186,
        25.78474239,  51.06063677,  78.86394581,  21.29825376,
        63.25317055,  25.95691679,  28.74760556,  41.67915888,
        85.61797327,  16.12536277, 144.97887605,   3.25411236,
       143.26558426,  10.80280311,  27.22940387,  32.16788881,
        22.40749618,  68.35865185,  22.02200086,  17.6461527 ,
        30.71776073,   6.84443329,  52.27788997,  17.57675762,
        11.94877864,   5.81463091,  42.41261208,  36.93191479,
        29.80877489,  10.17466521,  41.2171516 ,  11.58094957,
        21.56327735,   3.67121929,  21.46959995,  85.16300964,
        37.56942199,  52.77996747, 101.33686914, 105.63312288,
       108.88178651,  98.73462543,  29.30211134,  20.35339764,
        19.46589303,   3.7303917 ,  76.21329193,  31.93960953,
        15.91736342,   4.90339676,   5.75619879,  69.36156599,
        18.50684226,   1.58225504,  59.92660914,  12.62514187,
        16.19339716,  27.11780505,  30.83943269,   6.19

In [18]:
normalize(testres[0][:,np.newaxis], axis=0).ravel()

array([0.17533304, 0.45053436, 0.21928944, 0.01850934, 0.05381453,
       0.10656706, 0.16459448, 0.04445092, 0.13201372, 0.05417387,
       0.05999823, 0.08698727, 0.1786906 , 0.03365474, 0.30258088,
       0.00679156, 0.29900513, 0.02254619, 0.05682964, 0.0671366 ,
       0.04676599, 0.14266921, 0.04596143, 0.03682873, 0.06411008,
       0.0142848 , 0.10910755, 0.0366839 , 0.02493792, 0.01213553,
       0.08851804, 0.07707945, 0.06221296, 0.02123523, 0.08602303,
       0.02417024, 0.04500404, 0.00766209, 0.04480853, 0.17774106,
       0.07840997, 0.11015542, 0.21149701, 0.22046359, 0.22724378,
       0.20606595, 0.06115552, 0.04247894, 0.04062666, 0.00778558,
       0.15906238, 0.06666016, 0.03322063, 0.01023373, 0.01201358,
       0.14476236, 0.03862505, 0.00330228, 0.12507095, 0.02634954,
       0.03379673, 0.05659672, 0.06436402, 0.01292271, 0.0395671 ,
       0.05056007, 0.03769438, 0.04467789, 0.08899065, 0.01921684,
       0.01939903, 0.04660984, 0.00982715, 0.02140875, 0.06417

In [30]:
np.array(U_K.todense()[0])[0]

array([ 7, 35, 27,  1,  6,  5, 13,  6,  9,  0,  4,  7, 13,  6, 44,  1, 20,
        4,  7,  2,  7, 16,  8,  1,  4,  0, 11,  3,  0,  1, 16, 14,  4,  3,
        9,  2, 19,  2,  3, 13, 10,  5,  6, 22, 17,  5,  0, 11,  1,  1, 13,
        4,  1,  5,  4, 10,  4,  0, 15,  0,  7,  4, 12, 10,  7,  2,  1, 15,
        2,  0,  6,  0,  2,  2,  0])

In [29]:
normalize(np.array(U_K.todense()[0])[0][:,np.newaxis], axis=0).ravel()

array([0.07405438, 0.37027191, 0.28563833, 0.0105792 , 0.06347518,
       0.05289599, 0.13752957, 0.06347518, 0.09521278, 0.        ,
       0.04231679, 0.07405438, 0.13752957, 0.06347518, 0.46548469,
       0.0105792 , 0.21158395, 0.04231679, 0.07405438, 0.02115839,
       0.07405438, 0.16926716, 0.08463358, 0.0105792 , 0.04231679,
       0.        , 0.11637117, 0.03173759, 0.        , 0.0105792 ,
       0.16926716, 0.14810876, 0.04231679, 0.03173759, 0.09521278,
       0.02115839, 0.20100475, 0.02115839, 0.03173759, 0.13752957,
       0.10579197, 0.05289599, 0.06347518, 0.23274234, 0.17984636,
       0.05289599, 0.        , 0.11637117, 0.0105792 , 0.0105792 ,
       0.13752957, 0.04231679, 0.0105792 , 0.05289599, 0.04231679,
       0.10579197, 0.04231679, 0.        , 0.15868796, 0.        ,
       0.07405438, 0.04231679, 0.12695037, 0.10579197, 0.07405438,
       0.02115839, 0.0105792 , 0.15868796, 0.02115839, 0.        ,
       0.06347518, 0.        , 0.02115839, 0.02115839, 0.     

In [14]:
res = predict(U_K, 10, u_similarity)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:02<00:00, 2432.23it/s]


In [33]:
U_K[0][U_K[0].nonzero()]

matrix([[ 7, 35, 27,  1,  6,  5, 13,  6,  9,  4,  7, 13,  6, 44,  1, 20,
          4,  7,  2,  7, 16,  8,  1,  4, 11,  3,  1, 16, 14,  4,  3,  9,
          2, 19,  2,  3, 13, 10,  5,  6, 22, 17,  5, 11,  1,  1, 13,  4,
          1,  5,  4, 10,  4, 15,  7,  4, 12, 10,  7,  2,  1, 15,  2,  6,
          2,  2]])

## Creating a sterotype user u1

In [215]:
U_K = load_npz('../../data/beer/U_K.npz')
U_K = normalize(U_K)

In [239]:
key_phrase

,Phrases
0,roast
1,sweet
2,caramel
3,backbone
4,toast
5,bread
6,chocolate
7,biscuit
8,coffee
9,wheat


In [162]:
## All keyphrases
for i in key_phrase["Phrases"]:
    print i

roast
sweet
caramel
backbone
toast
bread
chocolate
biscuit
coffee
wheat
grain
yeast
citrus
toffee
fruit
nutty
bitter
barley
floral
honey
earthy
spic
smok
molass
sugar
 rye 
grass
cocoa
chew
butter
sour
 pine 
herbal
citric
grapefruit
resin
crisp
pungent
wood
orang
lemon
offwhite
 white
tan
brown
gold
beige
copper
mocha
khaki
black
yellow
watery
slick
silk
smooth
syrup
velvet
 red 
mahogany
ruby
tulip
cherry
plum
berry
banana
bourbon
oak
apple
metallic
raspberry
clove
apricot
coriander
pepper


In [243]:
key_phrase.index[key_phrase['Phrases'] == "beer"].tolist()

[]

In [164]:
row1_origin = U_K[0]

In [207]:
#row1_new
U_K[0,:] = 0

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:2: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  


In [208]:
#Add number 60, "ruby" to the keyphrase
U_K[0,60] = 1

In [197]:
print U_K

  (0, 0)	0.0
  (0, 1)	0.0
  (0, 2)	0.0
  (0, 3)	0.0
  (0, 4)	0.0
  (0, 5)	0.0
  (0, 6)	0.0
  (0, 7)	0.0
  (0, 8)	0.0
  (0, 9)	0.0
  (0, 10)	0.0
  (0, 11)	0.0
  (0, 12)	0.0
  (0, 13)	0.0
  (0, 14)	0.0
  (0, 15)	0.0
  (0, 16)	0.0
  (0, 17)	0.0
  (0, 18)	0.0
  (0, 19)	0.0
  (0, 20)	0.0
  (0, 21)	0.0
  (0, 22)	0.0
  (0, 23)	0.0
  (0, 24)	0.0
  :	:
  (6369, 20)	0.02596745720022887
  (6369, 21)	0.10386982880091548
  (6369, 22)	0.05193491440045774
  (6369, 24)	0.02596745720022887
  (6369, 30)	0.10386982880091548
  (6369, 34)	0.07790237160068661
  (6369, 36)	0.05193491440045774
  (6369, 39)	0.12983728600114436
  (6369, 41)	0.1817722004016021
  (6369, 42)	0.20773965760183097
  (6369, 43)	0.10386982880091548
  (6369, 44)	0.41547931520366194
  (6369, 45)	0.20773965760183097
  (6369, 47)	0.05193491440045774
  (6369, 50)	0.1817722004016021
  (6369, 51)	0.12983728600114436
  (6369, 54)	0.02596745720022887
  (6369, 55)	0.10386982880091548
  (6369, 56)	0.02596745720022887
  (6369, 58)	0.23370711480205

In [199]:
testres1 = predict(U_K, k, U_U_fromUI, item_similarity_en=False)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:02<00:00, 2500.00it/s]


In [202]:
print U_K

  (0, 0)	0.07405438192931117
  (0, 1)	0.3702719096465559
  (0, 2)	0.28563833029877167
  (0, 3)	0.010579197418473024
  (0, 4)	0.06347518451083815
  (0, 5)	0.05289598709236512
  (0, 6)	0.13752956644014933
  (0, 7)	0.06347518451083815
  (0, 8)	0.09521277676625722
  (0, 10)	0.042316789673892095
  (0, 11)	0.07405438192931117
  (0, 12)	0.13752956644014933
  (0, 13)	0.06347518451083815
  (0, 14)	0.4654846864128131
  (0, 15)	0.010579197418473024
  (0, 16)	0.21158394836946048
  (0, 17)	0.042316789673892095
  (0, 18)	0.07405438192931117
  (0, 19)	0.021158394836946048
  (0, 20)	0.07405438192931117
  (0, 21)	0.16926715869556838
  (0, 22)	0.08463357934778419
  (0, 23)	0.010579197418473024
  (0, 24)	0.042316789673892095
  (0, 26)	0.11637117160320327
  :	:
  (6369, 20)	0.02596745720022887
  (6369, 21)	0.10386982880091548
  (6369, 22)	0.05193491440045774
  (6369, 24)	0.02596745720022887
  (6369, 30)	0.10386982880091548
  (6369, 34)	0.07790237160068661
  (6369, 36)	0.05193491440045774
  (6369, 39)	0.12

In [187]:
# New_U_K * U_U_fromUI = U_K
testres2 = predict(U_K, k, U_U_fromUI, item_similarity_en=False)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:02<00:00, 2510.84it/s]


In [192]:
U_K.todense()[0]

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [203]:
testres1

array([[0.26401109, 0.7524112 , 0.32522061, ..., 0.03409106, 0.01808911,
        0.08343225],
       [0.28381758, 0.90050646, 0.43137004, ..., 0.01910784, 0.01170818,
        0.01970468],
       [0.27348309, 0.79198885, 0.30065618, ..., 0.03491223, 0.07341255,
        0.07485244],
       ...,
       [0.21064221, 1.09991938, 0.4337863 , ..., 0.03899793, 0.03914799,
        0.02654395],
       [0.20314659, 1.02733611, 0.28003989, ..., 0.01914292, 0.10212855,
        0.03587808],
       [0.38008604, 1.07139911, 0.37899053, ..., 0.04529136, 0.02162177,
        0.04132979]])

In [204]:
testres2

array([[0.26401109, 0.7524112 , 0.32522061, ..., 0.03409106, 0.01808911,
        0.08343225],
       [0.28381758, 0.90050646, 0.43137004, ..., 0.01910784, 0.01170818,
        0.01970468],
       [0.27348309, 0.79198885, 0.30065618, ..., 0.03491223, 0.07341255,
        0.07485244],
       ...,
       [0.21064221, 1.09991938, 0.4337863 , ..., 0.03899793, 0.03914799,
        0.02654395],
       [0.20314659, 1.02733611, 0.28003989, ..., 0.01914292, 0.10212855,
        0.03587808],
       [0.38008604, 1.07139911, 0.37899053, ..., 0.04529136, 0.02162177,
        0.04132979]])

In [216]:
a = U_U_fromUI* U_K 

In [211]:
b = U_U_fromUI* U_K 

In [217]:
a[0][60]

4.193095509454516

In [218]:
b[0][60]

5.119041458669164

Can see that not much changed since we are only utilizing user_1 and user_similarity matrix row 1

# U_U * U_K = U_K

How about do something in the U_U latent space (embedding)?

In [222]:
# Change U_I to get different U_U similarity
U_I = rtrain

In [224]:
print U_I

  (0, 33)	1.0
  (0, 124)	1.0
  (0, 128)	1.0
  (0, 148)	1.0
  (0, 160)	1.0
  (0, 325)	1.0
  (0, 481)	1.0
  (0, 595)	1.0
  (0, 605)	1.0
  (0, 679)	1.0
  (0, 793)	1.0
  (0, 830)	1.0
  (0, 970)	1.0
  (0, 1105)	1.0
  (0, 1190)	1.0
  (0, 1221)	1.0
  (0, 1278)	1.0
  (0, 1598)	1.0
  (0, 1809)	1.0
  (0, 2032)	1.0
  (0, 2064)	1.0
  (0, 2082)	1.0
  (0, 2144)	1.0
  (0, 2290)	1.0
  (0, 2502)	1.0
  :	:
  (6367, 2522)	1.0
  (6367, 3026)	1.0
  (6367, 3116)	1.0
  (6367, 3129)	1.0
  (6367, 3231)	1.0
  (6367, 3538)	1.0
  (6368, 132)	1.0
  (6368, 316)	1.0
  (6368, 967)	1.0
  (6368, 1137)	1.0
  (6368, 2174)	1.0
  (6368, 2953)	1.0
  (6368, 3460)	1.0
  (6369, 51)	1.0
  (6369, 1371)	1.0
  (6369, 1497)	1.0
  (6369, 1519)	1.0
  (6369, 1644)	1.0
  (6369, 1800)	1.0
  (6369, 2028)	1.0
  (6369, 2468)	1.0
  (6369, 2695)	1.0
  (6369, 3639)	1.0
  (6369, 3641)	1.0
  (6369, 3644)	1.0


but by changing/manipulating the U_I matrix, we need to know more information of each item. More specifically, we need to get a item-keyphrase matrix in order to change U_I as we want.

## I_K matrix

In [226]:
rtrain

<6370x3668 sparse matrix of type '<type 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [233]:
# I_K = get_I_K(df_train, row_name = 'ItemIndex', shape = (3668,75))
I_K = load_npz('../../data/beer/I_K.npz')

In [234]:
I_K

<3668x75 sparse matrix of type '<type 'numpy.int32'>'
	with 213696 stored elements in Compressed Sparse Row format>

In [237]:
print I_K[659]

  (0, 0)	2
  (0, 1)	67
  (0, 2)	41
  (0, 3)	12
  (0, 4)	21
  (0, 5)	32
  (0, 6)	1
  (0, 7)	32
  (0, 9)	3
  (0, 10)	21
  (0, 11)	10
  (0, 12)	86
  (0, 14)	56
  (0, 15)	3
  (0, 16)	95
  (0, 17)	1
  (0, 18)	48
  (0, 19)	27
  (0, 20)	16
  (0, 21)	24
  (0, 24)	5
  (0, 25)	1
  (0, 26)	31
  (0, 28)	4
  (0, 29)	8
  :	:
  (0, 35)	6
  (0, 36)	47
  (0, 37)	3
  (0, 38)	4
  (0, 39)	52
  (0, 40)	23
  (0, 41)	23
  (0, 42)	78
  (0, 43)	20
  (0, 45)	93
  (0, 47)	6
  (0, 51)	34
  (0, 52)	5
  (0, 53)	4
  (0, 54)	1
  (0, 55)	25
  (0, 56)	1
  (0, 57)	1
  (0, 61)	4
  (0, 63)	1
  (0, 64)	1
  (0, 68)	11
  (0, 69)	3
  (0, 72)	2
  (0, 74)	5


## Back to our problem

In [262]:
# I_K[:,0]: Items corresponding to Keyphrase 0
# Get top 100 items and reshape the matrix to array
keyphrase_id = 0
top_n = 100
np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[-top_n:]

array([2972, 2212, 2244, 2169,  746, 2666, 3472,  946, 2034, 2698, 2451,
       2607, 2053, 2708, 1195,  160, 2178,  879, 1800, 1433, 2710, 2052,
       3046, 1021, 1490, 2255, 1690,  656, 3118,  764,  332, 3121,  448,
       1088, 1374, 3280, 2275,  305,  979, 2250,  875, 1898,  108, 2912,
       2928,  358,  751, 2125,  684, 3152, 2366, 2198, 2217, 3252,  331,
         41, 2978,  371, 3606,  363,  569,   19,  453, 1871, 1249, 1373,
       2744, 1323, 1202,  740, 2707, 2701, 2073,  184,  409, 1647,  268,
        128, 3143, 3147, 1290, 1198, 2209,  374, 2923, 2691, 3628, 2732,
       3631, 1371,  364, 2925,  361, 2874, 2468,  749, 3240, 2215, 1639,
        597], dtype=int64)

In [273]:
print U_I[0,]

In [274]:
# Set some extreme U_I 
# For user1 set some items to 1
U_I[0,:] = 0
for i in np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[-top_n:]:
    U_I[0,i] = 1
# U_I[0][np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[-top_n:]] = 1

In [281]:
modified_U_U = train(U_I)

In [310]:
# Also set the User1_keyphrase to 1 for keyphrase_id
U_K[0,:] = 0
U_K[0,keyphrase_id] = 1

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:2: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  


In [294]:
a = normalize(modified_U_U*U_K)
print a

[[0.17001743 0.44794673 0.19390975 ... 0.01747935 0.01370377 0.03259141]
 [0.15974019 0.44848965 0.19490735 ... 0.01861396 0.01403853 0.03566853]
 [0.14626473 0.45303413 0.19573486 ... 0.02276955 0.01730831 0.03394567]
 ...
 [0.15046205 0.45345233 0.19705294 ... 0.01964526 0.01478368 0.03241993]
 [0.139078   0.4623413  0.1928427  ... 0.01817866 0.01462956 0.03138462]
 [0.15461639 0.45737168 0.18380551 ... 0.0171285  0.0140413  0.03020777]]


In [295]:
U_K = load_npz('../../data/beer/U_K.npz')
U_K = normalize(U_K)

In [296]:
c = normalize(U_U_fromUI*U_K)
print c

[[0.15290827 0.44002485 0.19897639 ... 0.01859834 0.01452302 0.0373506 ]
 [0.15898812 0.44854481 0.19493132 ... 0.01861625 0.01404025 0.03567291]
 [0.14626473 0.45303413 0.19573486 ... 0.02276955 0.01730831 0.03394567]
 ...
 [0.15046205 0.45345233 0.19705294 ... 0.01964526 0.01478368 0.03241993]
 [0.139078   0.4623413  0.1928427  ... 0.01817866 0.01462956 0.03138462]
 [0.15363115 0.45744283 0.1838341  ... 0.01713117 0.01404348 0.03021247]]


In [308]:
a[0]-c[0]

array([ 1.71091590e-02,  7.92188612e-03, -5.06663878e-03, -4.49576412e-03,
        2.74541551e-03, -8.32100201e-04,  1.35357631e-02, -2.52420651e-03,
        1.66367787e-02,  8.18038390e-03,  8.94047594e-03, -9.18837313e-03,
       -1.80837073e-02, -3.08653817e-03, -2.54688765e-02,  4.40090051e-03,
        1.10968215e-02,  1.58061405e-03, -2.41129293e-04, -1.34216968e-03,
       -7.11722229e-03, -2.88850162e-03,  4.57219612e-03, -2.23559302e-04,
       -7.72380811e-03, -2.21511548e-03, -4.62649471e-03, -9.32873900e-04,
        1.99022605e-03,  1.90004513e-03, -2.21641344e-02, -1.73565750e-02,
       -3.30229549e-03, -1.03600799e-03, -1.31841866e-02, -8.88344242e-03,
        8.08419646e-03, -1.60208218e-03, -6.59207977e-03, -1.30772370e-02,
       -8.71071807e-03, -1.58236708e-03, -7.96609662e-03,  6.50708811e-03,
        7.82693074e-03, -1.96684162e-03, -7.78590403e-04,  3.69055934e-03,
       -2.16818199e-04, -5.38291175e-04,  1.23422451e-02,  3.86455053e-03,
        8.60971785e-03, -

# User knn instead of matrix multiplication 

In [306]:
# Makes more sense tbh

In [312]:
a_ = predict(U_K, 100, modified_U_U)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2093.33it/s]


In [301]:
c_ = predict(U_K, 100, U_U_fromUI)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 2065.50it/s]


In [313]:
normalize(a_)[0]

array([0.204817  , 0.41073236, 0.21019623, 0.02714284, 0.07049075,
       0.08757557, 0.22480452, 0.02826776, 0.17256435, 0.04509956,
       0.07697874, 0.08722029, 0.15929047, 0.04006089, 0.27026299,
       0.02796288, 0.30935245, 0.04036599, 0.08312616, 0.04149064,
       0.04623904, 0.16130054, 0.06221338, 0.0433638 , 0.0707001 ,
       0.01064858, 0.05212279, 0.02815973, 0.03452487, 0.02329778,
       0.05453488, 0.0621945 , 0.02588329, 0.00753236, 0.05985014,
       0.02015382, 0.08721862, 0.00551488, 0.03602238, 0.14104107,
       0.05547301, 0.08576007, 0.17862484, 0.25611237, 0.2658404 ,
       0.13668961, 0.01760731, 0.07957436, 0.02732596, 0.00714931,
       0.1994329 , 0.0698211 , 0.03268207, 0.0218936 , 0.01280388,
       0.24472505, 0.04017002, 0.0116558 , 0.12447557, 0.021992  ,
       0.04515072, 0.02523078, 0.04271872, 0.02437379, 0.02961988,
       0.03217752, 0.02301125, 0.04462846, 0.05347196, 0.01260438,
       0.01052354, 0.03353737, 0.01759872, 0.01142327, 0.02779

In [305]:
normalize(c_)[0]

array([0.15707296, 0.41373421, 0.19636553, 0.03034219, 0.05356752,
       0.08854313, 0.1877936 , 0.03034999, 0.1268926 , 0.04013518,
       0.05588009, 0.11447344, 0.20743125, 0.04782659, 0.31785701,
       0.02023799, 0.31224246, 0.03440873, 0.08985072, 0.05231089,
       0.06714752, 0.15414929, 0.04233855, 0.02917779, 0.0750438 ,
       0.01615909, 0.05681413, 0.02420554, 0.02617858, 0.02041283,
       0.11086017, 0.08269542, 0.03674884, 0.01229725, 0.08051492,
       0.04592522, 0.0695531 , 0.01294599, 0.04299386, 0.17569948,
       0.07131478, 0.08565003, 0.20198444, 0.21604103, 0.24633009,
       0.15648577, 0.0228525 , 0.06694769, 0.02122285, 0.00477802,
       0.16328039, 0.08515915, 0.02333456, 0.02032284, 0.02124778,
       0.20300859, 0.05150626, 0.00997755, 0.13102778, 0.01366483,
       0.03398188, 0.07111063, 0.06607123, 0.02559484, 0.03698661,
       0.03542353, 0.04296371, 0.08641341, 0.07263242, 0.00874897,
       0.01342302, 0.03663887, 0.01802184, 0.01136855, 0.03528

In [314]:
normalize(a_)[0]-normalize(c_)[0]
# Makes more sense?

array([ 4.77440416e-02, -3.00185285e-03,  1.38306981e-02, -3.19934679e-03,
        1.69232291e-02, -9.67566754e-04,  3.70109164e-02, -2.08222707e-03,
        4.56717548e-02,  4.96437804e-03,  2.10986458e-02, -2.72531490e-02,
       -4.81407785e-02, -7.76570666e-03, -4.75940206e-02,  7.72489000e-03,
       -2.89001602e-03,  5.95726314e-03, -6.72455949e-03, -1.08202488e-02,
       -2.09084867e-02,  7.15125217e-03,  1.98748283e-02,  1.41860130e-02,
       -4.34370067e-03, -5.51051273e-03, -4.69134802e-03,  3.95418819e-03,
        8.34628956e-03,  2.88494751e-03, -5.63252835e-02, -2.05009133e-02,
       -1.08655470e-02, -4.76489384e-03, -2.06647762e-02, -2.57713967e-02,
        1.76655244e-02, -7.43110099e-03, -6.97147806e-03, -3.46584078e-02,
       -1.58417734e-02,  1.10043522e-04, -2.33596022e-02,  4.00713365e-02,
        1.95103124e-02, -1.97961661e-02, -5.24518867e-03,  1.26266699e-02,
        6.10310763e-03,  2.37129472e-03,  3.61525125e-02, -1.53380489e-02,
        9.34751349e-03,  

In [315]:
a[0]-c[0]

array([ 1.71091590e-02,  7.92188612e-03, -5.06663878e-03, -4.49576412e-03,
        2.74541551e-03, -8.32100201e-04,  1.35357631e-02, -2.52420651e-03,
        1.66367787e-02,  8.18038390e-03,  8.94047594e-03, -9.18837313e-03,
       -1.80837073e-02, -3.08653817e-03, -2.54688765e-02,  4.40090051e-03,
        1.10968215e-02,  1.58061405e-03, -2.41129293e-04, -1.34216968e-03,
       -7.11722229e-03, -2.88850162e-03,  4.57219612e-03, -2.23559302e-04,
       -7.72380811e-03, -2.21511548e-03, -4.62649471e-03, -9.32873900e-04,
        1.99022605e-03,  1.90004513e-03, -2.21641344e-02, -1.73565750e-02,
       -3.30229549e-03, -1.03600799e-03, -1.31841866e-02, -8.88344242e-03,
        8.08419646e-03, -1.60208218e-03, -6.59207977e-03, -1.30772370e-02,
       -8.71071807e-03, -1.58236708e-03, -7.96609662e-03,  6.50708811e-03,
        7.82693074e-03, -1.96684162e-03, -7.78590403e-04,  3.69055934e-03,
       -2.16818199e-04, -5.38291175e-04,  1.23422451e-02,  3.86455053e-03,
        8.60971785e-03, -

# Critiquing Draft

##### TF-IDF to get U_K